In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas_profiling as pp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
import seaborn as sns
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, roc_curve, auc, confusion_matrix, plot_roc_curve, accuracy_score
from sklearn import preprocessing

In [ ]:

raw_data = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(raw_data.dtypes)
pd.set_option('display.max_columns', None)
raw_data.head()

In [ ]:
raw_data.isnull().sum()

Existem apenas 3 variáveis numéricas (SeniorCitizen, tenure e MonthlyCharges) e muitas variáveis categóricas. Aparentemente não tem NaN ou missing data. Quero primeiro ver se existe alguma relação clara entre alguma variável e churn e depois tentar visualizar a combinação de algumas variáveis e o churn. Também vou procurar por outliers e ver a distribuição das duas variáveis numéricas.

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
        'Contract', 'PaperlessBilling', 'PaymentMethod']

plt.subplots(4, 4, sharex=True, sharey=True, figsize=(15,15))
i = 1
for column in cols:
    plt.subplot(4, 4, i)
    plt.title(column)
    raw_data[column].value_counts(normalize=True).sort_values().plot(kind = 'barh', fontsize=9, sharex=False)
    i = i + 1
plt.show()



# Perfil do Cliente 

*  Os produtos extras (além da linha de telefone e internet) mais consumidos parecem ser Streaming TV e Streaming Movies, com aproximadamente 40% do total dos clientes.
*  Os outros produtos, Backup Online e Device PRotection, menos de 40% dos clientes usam e Tech Support menos de 30%.
*  A maioria dos contratos (mais de 50%) são mês a mes e mais de 60% paperless
*  O método de pagamento mais usado é eletronick check. Os outros 3 métodos restantes são similares (20% cada)
* Das variáveis sociais, apenas dependentes me chamou a atenção. Mais de 60% não possuem, embora quase 50% possuam conjuge, então talvez não valha a pena pensar em produtos pra família ou crianças.

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
        'Contract', 'PaperlessBilling', 'PaymentMethod']

plt.subplots(4, 4, sharex=True, sharey=True, figsize=(15,15))
i = 1
for column in cols:
    plt.subplot(4, 4, i)
    plt.title(column)
    raw_data[ raw_data['Churn'] == 'Yes'][column].value_counts(normalize=True).sort_values().plot(kind = 'barh', fontsize=9)
    i = i + 1
plt.show()

# Perfil dos Clientes onde Churn = Yes

*   O percentual dos clientes com contrato de dois anos ou um ano que dão churn é menor que o percentual total de clientes com esses contratos. Talvez esse modo de contrato tenha alguma multa ou alguma vantagem para o cliente permanecer com o produto.
*    A maioria dos clientes não possuem serviços extras como Streaming. Talvez clientes que compram o streaming veja mais valor no produto e decidam permanecer ou quem desiste são pessoas que tem um plano de serviços muito básico e mesmo assim não conseguem pagar.
*    80% não possuem tech support, talvez eles não consigam resolver problemas e decidem encerrar a conta. Uma sugestão pra tentar reduzir o churn é fornecer algum nível de suporte técnico gratuito.
*    Em relação as variáveis sociais: a) a proporção de senior que dá churn é maior que a proporção de seniors clientes b) a proporção de clientes sem conjuge é maior também e c) a proporção de clientes sem dependentes tbm é maior. Então talvez valha a pena fazer produtos pra família pra tentar reduzir o churn.

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
        'Contract', 'PaperlessBilling', 'PaymentMethod']

fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(15,15))
#fig.subplots_adjust(left = 0.4)
i = 1
for column in cols:
    plt.subplot(4, 4, i)
    plt.title(column)
    raw_data[ raw_data['Churn'] == 'No'][column].value_counts(normalize=True).sort_values().plot(kind = 'barh', fontsize=9, sharex=True)

    i = i + 1
plt.show()

# Perfil Churn = No

Esse perfil é complementar ao anterior e eu só fiz por motivos de completude. Note como contratos do tipo two year and one year aparecem em uma proporção maior entre os que não dão churn.


In [ ]:
a = sns.jointplot(x="tenure", y="MonthlyCharges", data=raw_data[raw_data['Churn']=='Yes'], kind="kde", height=5)
plt.title('Churn Yes')
b = sns.jointplot(x="tenure", y="MonthlyCharges", data=raw_data[raw_data['Churn']=='No'], kind="kde", height=5)
plt.title('Churn No')

# Variáveis númericas e Churn

Os dois gráficos acima mostram:
*    O churn alto ocorre em mensalidades elevadas e em pouco tempo.
*    Os clientes que não dão churn estão em mensalidades baratas. Note que a distribuição de churn = no para o preço parece ser bimodal. Provavelmente porque essa faixa de preço consegue fidelizar clientes e talvez essa variável seja algo como: se tenure > 70 tenure = 70
*    Existem alguns clientes que fidelizaram mesmo pagando muito, acho que vale a pena ver o perfil desses clientes.

In [ ]:
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 
        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
        'Contract', 'PaperlessBilling', 'PaymentMethod']

fig, axes = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(15,15))
#fig.subplots_adjust(left = 0.4)
i = 1
for column in cols:
    plt.subplot(4, 4, i)
    plt.title(column)
    raw_data[ (raw_data['Churn'] == 'No') & (raw_data['tenure'] > 40) & (raw_data['MonthlyCharges'] > 60)  ][column].value_counts(normalize=True).sort_values().plot(kind = 'barh', fontsize=9, sharex=True)

    i = i + 1
plt.show()

# Perfil dos clientes fidelizados

*     Os clientes fidelizados (com tenure > 40) e MonthlyCharge > 60 compram todos os produtos, pagam com métodos automáticos, possuem múltiplas linhas, dependentes e conjuge.

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(pd.get_dummies(raw_data.loc[:, raw_data.columns != 'Churn']))
tsne_df = pd.DataFrame(data=X_embedded, columns=["X", "Y"])
tsne_df['Churn'] = raw_data['Churn']
sns.scatterplot(data=tsne_df, x="X", y="Y", hue="Churn", Alpha = 0.5)

# Projeção dos dados

O gráfico acima mostra os dados dos clientes projetados em um plano onde a cor informa o padrão de churn. Podemos ver que existem agrupamentos de clientes onde acontece mais churn e agrupamentos com menos churn, então é provavel que os métodos de aprendizagem de máquina sejam capazes de encontrar um hiperplano que separe as duas classes.

# Modelagem

Vou testar diversos modelos e como temos um problema de classificação binário vou usar a matriz de confusão como métrica para discussão do ajuste do modelo.

In [ ]:
# tem duas variáveis numericas que podemos criar categorias: tenure e MonthlyCharges, vou fazer isso mas manter também a variável numérica..
new_df = pd.DataFrame()
#coloquei 12 pra ter a informação do churn de 6 em 6 meses, talvez valha a pena aumentar (por exemplo pra 3 em 3 meses)
new_df['tenure_cat'] = pd.cut(raw_data['tenure'], 12, labels=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] ) 
#pelo gráfico da densidade do ChurnxTenurexCusto parece ter 3 ou 4 categorias do MonthlyCharges, talvez valesse a pena fazer um ajuste fino maior, mas vou cortar em 5
new_df['charges_cat'] = pd.cut(raw_data['MonthlyCharges'], 5, labels=['1', '2', '3', '4', '5'] ) 

# essas colunas possuem um valor repetido em todas, que é o valor No Internet Service, mas esse valor já está descrito na coluna InternetService
# então eu separei as colunas que já possuem essa informação de No Internet Service repetido para remove-las depois do dummies
col_dummies_repeated = ['OnlineSecurity', 'OnlineBackup',  'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies' ]
dummies = pd.get_dummies( raw_data[col_dummies_repeated ] ) 
dummies = dummies.drop( columns=['OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 'DeviceProtection_No internet service', 'TechSupport_No internet service', 'StreamingTV_No internet service', 'StreamingMovies_No internet service'] )

col_dummies = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'Contract', 'PaperlessBilling', 'PaymentMethod']
dummies2 = pd.get_dummies( raw_data[col_dummies ], drop_first=True)

#concatenando os dummies
new_df = pd.concat([new_df, dummies], axis=1)
new_df = pd.concat([new_df, dummies2], axis=1)

#pegando os dummies das variaveis categoricas que foram criadas
new_df = pd.get_dummies(new_df, drop_first=True )

#adicionando as duas colunas numericas

new_df['tenure'] = raw_data['tenure']
new_df['MonthlyCharges'] = raw_data['MonthlyCharges']

new_df['Churn'] = raw_data['Churn']
new_df = new_df.replace({'Yes': 1, 'No': 0})


In [ ]:
new_df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( new_df.loc[:, new_df.columns != 'Churn'], new_df['Churn'], test_size=0.2, random_state=42)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
parameters = {'n_estimators':[10, 20, 50, 100], 'criterion':['gini', 'entropy'], 'max_features':['auto', 'sqrt', 'log2'] }
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters, n_jobs=4)
clf.fit(X_train, y_train)
print('Best Params:', clf.best_params_)
y_pred = clf.predict(X_test)
m = confusion_matrix(y_test, y_pred)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test)

In [ ]:
clf_score = cross_val_score(rf, X_train, y_train, cv=10)
print(clf_score)
clf_score.mean()

In [ ]:
features = pd.DataFrame(clf.best_estimator_.feature_importances_)
features["Feature"] = list(new_df.columns[:-1]) 
features.sort_values(by=0, ascending=False).head()


g = sns.barplot(0,"Feature",data = features.sort_values(by=0, ascending=False)[0:10], palette="Pastel1",orient = "h")
g.set_xlabel("Weight")
g = g.set_title("Random Forest")

# Modelo Random Forest

O modelo teve uma acurácia em torno de 78% mas errou bastante a classe mais difícil de prever, que é Churn = 1. Aproveitei para plotar o peso das variáveis que o modelo encontrou e como esperado, o que pesou mais foram as variáveis numéricas seguido por OnlineSecurity_No e Contract_Two_year. 

In [ ]:
parameters = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'C':[0.9, 1.0, 1.1] }
svm = SVC()
clf = GridSearchCV(svm, parameters, n_jobs=4)
clf.fit(X_train, y_train)
print('Best Params:', clf.best_params_)
y_pred = clf.predict(X_test)
m = confusion_matrix(y_test, y_pred)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test)

In [ ]:
clf_score = cross_val_score(svm, X_train, y_train, cv=10)
print(clf_score)
clf_score.mean()

# Modelo SVM

A SVM tem um resultado melhor, com o melhor modelo acertando 81% e isso se refletiu em uma ligeira redução nos erros tipo I e tipo II. Nesse caso o melhor tipo de kernel retornado foi o linear e não o rbf, que é melhor para funções não lineares.

In [ ]:
parameters = {'penalty':['l1', 'l2', 'elasticnet', 'none'], 'C':[0.9, 1.0, 1.1] }
reglog = LogisticRegression()
clf = GridSearchCV(reglog, parameters, n_jobs=4)
clf.fit(X_train, y_train)
print('Best Params:', clf.best_params_)
y_pred = clf.predict(X_test)
m = confusion_matrix(y_test, y_pred)

In [ ]:
clf_score = cross_val_score(reglog, X_train, y_train, cv=10)
print(clf_score)
clf_score.mean()

In [ ]:
plot_confusion_matrix(clf, X_test, y_test)

In [ ]:
pred_proba_df = pd.DataFrame(clf.best_estimator_.predict_proba(X_test))
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
    test_accuracy = accuracy_score(y_test, Y_test_pred.iloc[:,1].to_numpy())
    print('Our testing accuracy is {}'.format(test_accuracy))

    print(confusion_matrix(y_test, Y_test_pred.iloc[:,1].to_numpy()))


# Modelo Regressão Logística

A regressão logística teve uma acurácia similar à SVM mas teve uma ligeira inversão nos erros, com a regressão logística acertando mais os casos de churn mas errado mais os casos de não churn. Note que ambos deram acurácia similar, mas como é mais importante pro negócio acertar o maior numéro possível de casos de churn, modelos que consigam isso provavelmente seriam escolhidos.

No último bloco de código eu variei o limiar da probabilidade da classificação. Podemos reduzir bastante o erro do churn = 1 se estivermos dispostos a aceitar um erro maior de classificar churn = 0 como = 1. Pode ser uma alternativa, por exemplo, caso o modelo identifique uma chance de churn podemos tentar alguma intervenção com o cliente, como oferecer algum serviço gratuito por um período de tempo. Por exemplo, como na análise exploratória vimos que quem tem streaming de filmes tem menos churn, pode ser que valha a pena oferecer alguns meses do serviço gratuitamente ou um upgrade nos serviçs com desconto. O limiar de 25% faz com que acertamos 313 churns e erramos 60, mas teriamos 292 falso positivos. Se os custos da intervenção desses 292 for menor que o custo do churn, então pode valer a pena.

# Conclusão sobre a modelagem

Apesar de eu ainda achar que os modelos poderiam ser melhorados, por exemplo, com um gridsearch melhor, trabalhando mais as variáveis (repensando os pontos de corte das variáveis numéricas, tentando combinações e removendo variáveis) e testando mais modelos e tentar estratégias para datasets desbalanceados.